In [1]:
import numpy as np
    

In [2]:
def dist(p1, p2):
    #return np.sqrt(np.sum(np.square(np.array(p1) - np.array(p2))))
    return np.sum(np.abs(np.array(p1) - np.array(p2)))

In [3]:
infty = 10000000
def optimal_2server(x1,x2,req):
    full = np.concatenate((np.array([x1,x2]) , req),  )
    l = len(full)
    dp = [[ infty for i in range(l+1)] for j in range(l+1)]
    dp[1][0]  = 0
    
    for i in range(1,l-1):
        for j in range(i):
            dp[i+1][j] = min(dp[i+1][j], dp[i][j] + dist(full[i+1], full[i]))
            dp[i+1][i] = min(dp[i+1][i], dp[i][j] + dist(full[i+1], full[j]))
            
                        
    ans =  infty
    for i in range(l-1):
            ans = min(ans, dp[l-1][i])
    return ans    
            
    
def optimal_3server(x1, x2, x3, req):
    full = np.concatenate((np.array([x1,x2,x3]), req), )
    l = len(full)
    dp = [[[ infty for i in range(l+1)] for j in range(l+1)] for k in range(l+1)]
    dp[2][1][0]  = 0
    
    for i in range(2,l-1):
        for j in range(i):
            for k in range(j):
                dp[i+1][j][k] = min(dp[i+1][j][k], dp[i][j][k] + dist(full[i+1], full[i]))
                dp[i+1][i][k] = min(dp[i+1][i][k], dp[i][j][k] + dist(full[i+1], full[j]))
                dp[i+1][i][j] = min(dp[i+1][i][j], dp[i][j][k] + dist(full[i+1], full[k]))
                        
    ans =  infty
    for i in range(l-1):
        for j in range(l-1):
            ans = min(ans, dp[l-1][i][j])
    return ans    
    
    

In [4]:
def closest_2server(x1, x2, req):
    l = len(req)
    pos1 = x1
    pos2 = x2
    total = 0
    for i in range(l):
        if(dist(pos1, req[i]) < dist(pos2, req[i])):
            total += dist(pos1, req[i])
            pos1 = req[i]
        else:
            total += dist(pos2 , req[i])
            pos2 = req[i]
    return total

def closest_3server(x1,x2,x3, req):
    l = len(req)
    pos1 = x1
    pos2 = x2
    pos3 = x3
    total = 0
    for i in range(l):
        shortest = min(dist(pos1, req[i]), dist(pos2, req[i]), dist(pos3, req[i]))
        if(dist(pos1, req[i]) == shortest ):
            total += dist(pos1, req[i])
            pos1 = req[i]
        elif (dist(pos2, req[i]) == shortest ):
            total += dist(pos2, req[i])
            pos2 = req[i]
        else:
            total += dist(pos3, req[i])
            pos3 = req[i]
    return total
            

def oneserver(x1, req):
    l = len(req)
    pos = x1
    total = 0
    for i in range(l):
        total += dist(pos, req[i])
        pos = req[i]
    return total
        

In [34]:
dim = 2

def optimal_online(x1,x2,req, monte_carlo = 50, lookahead = 5):
    if(len(req) == 0):
        return 0
    else:
        total1 = 0
        total2 = 0
        for i in range(monte_carlo):
            test = np.random.normal(0,1, (lookahead,dim))
            total1 += (optimal_2server(x2,req[0],test) + dist(x1,req[0]))
            total2 += (optimal_2server(x1,req[0],test) + dist(x2,req[0]))
        if(total1 < total2):
            return optimal_online(x2,req[0],req[1:]) + dist(x1,req[0])
        else:
            return optimal_online(x1,req[0],req[1:]) + dist(x2,req[0])
            
    
    

In [36]:

samples = 100
iterations = 100
origin = np.zeros(dim)

opt = 0
clos = 0
oneserv = 0
split_in_half = 0
opt_online = 0


for i in range(iterations):
    s = np.random.normal(0,1 , (samples, dim))
    #s = []
    #for j in range(samples):
        #ind = np.random.randint(-1,2)
        #s.append([np.random.normal(0,1)])
    #s = np.array(s)
        
    opt += optimal_2server(origin,origin,  s)
    clos += closest_2server(origin,origin,  s)
    oneserv += oneserver(origin,s)
    opt_online += optimal_online(origin, origin, s)
    
    #s_pos = []
    #s_neg = []
    #for j in range(samples):
        #if(s[j] > 0):
            #s_pos.append(s[j])
        #else:
            #s_neg.append(s[j])
    #s_pos = np.array(s_pos)
    #s_neg = np.array(s_neg)
    #split_in_half += (oneserver(origin, s_pos) + oneserver(origin, s_neg))
    if(i%10 ==0):
        print(i/10)

print(clos/opt)
print(oneserv/opt)
print(split_in_half/opt)
print(opt_online/opt)


0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
1.1192981464805907
1.4793930715153605
0.0
1.0753978181503876
